In [ ]:
# This file is part of pydidas.
#
# Copyright 2023 - 2024, Helmholtz-Zentrum Hereon
# SPDX-License-Identifier: GPL-3.0-only
#
# pydidas is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License version 3 as
# published by the Free Software Foundation.
#
# Pydidas is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Pydidas. If not, see <http://www.gnu.org/licenses/>.

"""
The dataset module includes the Dataset subclasses of numpy.ndarray with additional
embedded metadata.
"""

__author__ = "Gudrun Lotze"
__copyright__ = "Copyright 2024, Helmholtz-Zentrum Hereon"
__license__ = "GPL-3.0-only"
__maintainer__ = "Malte Storm, Gudrun Lotze"
__status__ = "Development"

In [11]:
import os
import h5py as h5
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm
import matplotlib.cm as cm
from scipy.optimize import curve_fit
from pydidas.core import Dataset
from pydidas.data_io import import_data
%matplotlib inline


import importlib

fname = '/Users/lotzegud/pydidas/pydidas_plugins/proc_plugins/stress_strain.py'
spec = importlib.util.spec_from_file_location('stress_strain', fname)
stress_strain = importlib.util.module_from_spec(spec)
spec.loader.exec_module(stress_strain)

# Check if the module object is created
if hasattr(stress_strain, '__file__'):
    print("Module was successfully loaded.")

# Inspect module attributes
print("Module attributes:", dir(stress_strain))

#We need to append the path to the sys.path to import the module
sys.path.append('/Users/lotzegud/pydidas/pydidas_plugins/proc_plugins')
from  stress_strain import *
print(stress_strain)

extract_d_spacing()



Module was successfully loaded.
Module attributes: ['Dataset', '__author__', '__builtins__', '__cached__', '__copyright__', '__doc__', '__file__', '__license__', '__loader__', '__maintainer__', '__name__', '__package__', '__spec__', '__status__', 'chi_pos_verification', 'combine_sort_d_spacing_pos_neg', 'connected_components', 'csr_matrix', 'ds_slicing', 'extract_d_spacing', 'group_d_spacing_by_chi', 'h5', 'idx_s2c_grouping', 'import_data', 'np', 'os']
<module 'stress_strain' from '/Users/lotzegud/pydidas/pydidas_plugins/proc_plugins/stress_strain.py'>


TypeError: extract_d_spacing() missing 3 required positional arguments: 'ds1', 'pos_key', and 'pos_idx'

Auxiliary functions

In [ ]:
def chi_gen(chi_start, chi_stop, delta_chi):
    if chi_start >= chi_stop:
        raise ValueError('chi_start has to be smaller than chi_stop')
    return np.arange(chi_start, chi_stop, delta_chi)


def predefined_metric_calculation(metric_name, chi, x, y, d0, spatial_var, phase_shift):
    """ Calculate predefined metric based on name, applying spatial variation even if x is not provided. """
    # Handle spatial variation by introducing a default or random x if none is provided
    if x is None and spatial_var:
        x = np.random.uniform(0, 1)  #A random x between 0 and 5
    if metric_name == "position":
        return 0.2832*np.sin(np.deg2rad(chi+phase_shift))**2 + d0 + (0.01 * x if spatial_var else 0)
    if metric_name == "area":
        return np.random.uniform(6, 37, size=len(chi)) + 0.1 * y
    if metric_name == "FWHM":
        return np.random.uniform(0.35, 0.75, size=len(chi))
    if metric_name == "background at peak":
        return np.random.uniform(2.3, 5.3, size=len(chi))
    if metric_name == "total count intensity":
        return np.random.uniform(80, 800, size=len(chi))
    return np.random.uniform(1.5708, 3.141, size=len(chi))  # Fallback for unknown metrics

def generate_spatial_fit_res(y_range, x_range=None, chi_start=-175, chi_stop=180, delta_chi=10, fit_labels=None, spatial_var=True, phase_shift=0):
    '''
    chi [degree]
    phase_shift [degree]
    '''
    
    if fit_labels is None:
        fit_labels = '0: position; 1: area; 2: FWHM; 3: background at peak; 4: total count intensity'
    fit_labels_dict = {int(k.split(':')[0].strip()): k.split(':')[1].strip() for k in fit_labels.replace(', ', ';').split(';')}

    chi = chi_gen(chi_start, chi_stop, delta_chi)
    d0 = 25  # in nm

    # Determine the dimensions based on x_range
    if x_range is not None:
        result_array = np.empty((len(y_range), len(x_range), len(chi), len(fit_labels_dict)))
    else:
        result_array = np.empty((len(y_range), len(chi), len(fit_labels_dict)))
        x_range = [None]  # Simulate the absence of x values

    # Perform calculations for each y and x, and across all metrics
    for j, y in enumerate(y_range):
        for i, x in enumerate(x_range):
            fit_results = []
            for idx in sorted(fit_labels_dict.keys()):
                metric_name = fit_labels_dict[idx]
                result = predefined_metric_calculation(metric_name, chi, x, y, d0, spatial_var, phase_shift)
                fit_results.append(result)

            fit_results = np.array(fit_results)
            # Adjust how results are stored based on the presence of x_range
            # Debug print statements
            #print(f"fit_results.T.shape: {fit_results.T.shape}, j: {j}, i: {i}")
            #print('x_range:', x_range)
            if x is not None:
                result_array[j, i, :, :] = fit_results.T
            else:
                result_array[j, :, :] = fit_results.T  # Ensure dimensionality matches expected (len(chi), len(fit_labels_dict))

    return result_array

def adding_noise_d_spacing(ds1, scaling=7e-2):
    '''
    ds [Dataset], two-dimensional Dataset, expecting in first column d-spacing values.
    '''
    #Introducing seed and random noise for d_spacing
    rng = np.random.default_rng(seed=9973)
    ds1[:,0]+=(0.5-rng.random(ds1.shape[0]))*scaling
    
    return ds1   

def plot_d_spacing_vs_chi(result_array, chi, positions):
    """
    Plots d_spacing vs chi for specified (x, y) positions in the result array using Matplotlib's OOP interface.

    Parameters:
        result_array (numpy.ndarray): The 4D array containing measurement data.
        chi (numpy.ndarray): The array of chi values.
        positions (list of tuples): A list of (x_index, y_index) tuples specifying the positions to plot.
    """
    
    fig, ax = plt.subplots()
    for (x_index, y_index) in positions:
        # Extract d_spacing for the specific position
        d_spacing = result_array[x_index, y_index, :, 0]  # d_spacing is the first property in the last dimension
        
        # Plotting using the axes object
        ax.plot(chi, d_spacing, label=f'(x={x_index}, y={y_index})', marker='o', linestyle='--')
    
    ax.set_xlabel('chi [deg]')
    ax.set_ylabel('d_spacing')
    ax.set_title('d_spacing vs chi for various x,y')
    ax.grid(True)
    
    ax.legend()
    plt.show()

In [ ]:

    
    
def chi_pos_verification(ds):
    '''
    Verification if dataset ds contains 'chi' and 'position' for d-spacing.
    Returns:
        chi_key: The index associated with 'chi'.
        position_key: A tuple where the first element is the index in axis_labels where 'position' descriptor is found, and the second element is the key in the structured string resembling a dict.    
    '''
    if not isinstance(ds, Dataset):
        raise TypeError('Input has to be of type Dataset.')
        
    axis_labels=ds.axis_labels
    
    # Collect indices where 'chi' is found
    chi_indices = [key for key, value in axis_labels.items() if value == 'chi']

    # Check for multiple 'chi'
    if len(chi_indices) > 1:
        raise KeyError('Multiple "chi" found. Check your dataset.')

    # Check for absence of 'chi'
    if not chi_indices:
        raise ValueError('chi is missing. Check your dataset.')

    # Assuming there's exactly one 'chi', get the index
    chi_key = chi_indices[0]

    reverse_axis_labels = dict(zip(axis_labels.values(), axis_labels.keys()))

    # Process to find 'position' in the complex structured string
    position_key = None
    for value, position_index in reverse_axis_labels.items():
        if isinstance(value, str) and 'position' in value:
            parts = value.split(';')
            for part in parts:
                if 'position' in part:
                    # Assume the part is structured as 'key: description'
                    part_key, _ = part.split(':')
                    part_key = int(part_key.strip())  # Convert the key part to integer
                    position_key = (position_index, part_key)
                    break
            if position_key is not None:
                break

    # Check if 'position' is found
    if position_key is None:
        raise ValueError('Key containing "position" is missing. Check your dataset.')

    return (chi_key, position_key)


def extract_d_spacing(ds1, pos_key, pos_idx):
    '''
    Extracts d-spacing
    
    Parameters: 
    - ds1 (Dataset): A Dataset object
    - pos_key (int): Key containing 'position' information
    - pos_idx (int): Index containing 'position' information in substring 
    
    '''    
    _slices = []
    for _dim in range(ds1.ndim):
        if _dim != pos_key:
            _slices.append(slice(None, None))
        elif _dim == pos_key:
            _slices.append(slice(pos_idx, pos_idx + 1))
        #print(f"Dimension {_dim}, Slices: {_slices}")
        
    d_spacing = ds1[*_slices]
    d_spacing = d_spacing.squeeze()
        
    return d_spacing
    
def ds_slicing(ds1):
    '''
    Extracts d-spacing values and associated chi values from a Dataset object for one scan position.

    Parameters:
    - ds1 (Dataset): A Dataset object. 

    Returns:
    - chi (array-like): Array of chi values associated with the extracted d-spacing values.
    - d_spacing (array-like): Array of d-spacing values extracted from the Dataset object.

    Raises:
    - TypeError: If the input is not of type Dataset.
    - ValueError: If the dimension of the d_spacing is not 1.
    '''

    if not isinstance(ds1, Dataset):
        raise TypeError('Input has to be of type Dataset.')
      
    chi_key, (pos_key, pos_idx) = chi_pos_verification(ds1)
    
    #select the chi values
    chi=ds1.axis_ranges[chi_key]
    
    # Extract d-spacing values
    d_spacing = extract_d_spacing(ds1, pos_key, pos_idx)
    
    if d_spacing.size == 0: 
        #Should check for empty arrays in case of slicing beyond bounds
        raise ValueError('Array is empty.')
    
    if not d_spacing.ndim == 1: 
        raise ValueError('Dimension mismatch.')
                       
    return chi, d_spacing

def main():
    
    #Creating of dummy data
    delta_chi =10
    chi_start= 0
    chi_stop= 90
    phase_shift = 70
    chi=chi_gen(chi_start, chi_stop, delta_chi)
    print(chi)
    print('Laenge chi', len(chi))

    #x, y in um
    y = np.arange(2, 8)
    x = np.arange(0, 5)

    #labels
    fit_labels= '0: position; 1: area; 2: FWHM; 3: background at peak; 4: total count intensity'
    fit_labels_dict = {int(item.split(":")[0].strip()): item.split(":")[1].strip() for item in fit_labels.split(";")}
    num_labels = len(fit_labels_dict)
    
    #creation of Pydidas Dataset
    axis_labels= ['y', 'x', 'chi', fit_labels]
    axis_ranges = {0: y, 1:x, 2: chi , 3: np.arange(num_labels)} 
    
   
    result_array_spatial = generate_spatial_fit_res(y, x, chi_start,chi_stop, delta_chi, fit_labels , spatial_var=True, phase_shift=phase_shift)
    #result_array = generate_spatial_fit_res(y, x, chi_start,chi_stop, delta_chi, fit_labels , spatial_var=False)
    print('Result array spatial', result_array_spatial.shape)

    
    #visualisation
    #positions = [(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
    #plot_d_spacing_vs_chi(result_array_spatial, chi, positions)
    #plot_d_spacing_vs_chi(result_array, chi, positions)


    ds = Dataset(result_array_spatial,  axis_labels=axis_labels, axis_ranges=axis_ranges)
    axis_labels = ds.axis_labels
    print(axis_labels)
    print(ds.shape)
    
    #choose random location in ds
    y_idx = 5
    x_idx = 3
    
    # slice Dataset based on location
    ds1 = ds[y_idx, x_idx]
      
    print('Shape of ds1 dataset before ', ds1.shape)
    
    #Introducing seed and random noise for d_spacing
    ds1 = adding_noise_d_spacing(ds1, scaling=4e-2)
    
    #Introducing np.nan 
    #print('Content ds1 array',  ds1.array[:,0])
    #ds1.array[0:9:2,0] = np.nan
    #ds1.array[18:28:2,0] = np.nan
        
    print('Shape of ds1 dataset ', ds1.shape)
    print('Content ds1 array after',  ds1.array[:,0])
      
    chi_key, (pos_key, pos_idx) = chi_pos_verification(ds1)
    print(chi_key, pos_key, pos_idx)
    
    chi, d_spacing = ds_slicing(ds1)    
    
    d_spacing_pos, d_spacing_neg=group_d_spacing_by_chi(d_spacing, chi)
    print('s2c_pos', d_spacing_pos.axis_ranges[0])
    print('d_spacing_pos', d_spacing_pos.array)
    print('s2c_neg', d_spacing_neg.axis_ranges[0])
    print('d_spacing_neg', d_spacing_neg.array)
    
    d_spacing_combined = combine_sort_d_spacing_pos_neg(d_spacing_pos, d_spacing_neg)
    print(d_spacing_combined) 
    
    d_diff= np.diff(d_spacing_combined.array, axis=0).squeeze()
    x_diff = np.sin(2*np.arcsin(np.sqrt(d_spacing_combined.axis_ranges[1])))
    
    print('d_diff', d_diff)
    print('x_diff', x_diff)
    print(d_spacing_combined.axis_ranges[1])
    
    fig, ax = plt.subplots()
    ax.plot(x_diff, d_diff, linestyle='None', marker='o')
    ax.set_title('d(+) - d(-) vs sin(2*chi)')
    ax.set_xlabel('sin(2*chi)')
    ax.set_ylabel('d(+) - d(-) [nm]')
    ax.grid()
    fig.show()
    

if __name__ == "__main__":
    main()